In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install Augmentor

In [4]:
import Augmentor
import os
import shutil
from PIL import Image
import numpy as np

Documentation on Augmentor can be found here: https://github.com/mdbloice/Augmentor

In [5]:
pathA = '/content/drive/My Drive/DigitalFutures Workshop/Dataset/A'
pathB = '/content/drive/My Drive/DigitalFutures Workshop/Dataset/B' 

In [12]:
p = Augmentor.Pipeline(pathA)
# Point to a directory containing ground truth data.
# Images with the same file names will be added as ground truth data
# and augmented in parallel to the original data.
p.ground_truth(pathB)

# Add operations to the pipeline:
p.rotate90(probability=0.25)
p.rotate270(probability=0.25)
p.flip_left_right(probability=0.25)
p.flip_top_bottom(probability=0.25)


Initialised with 104 image(s) found.
Output directory set to /content/drive/My Drive/DigitalFutures Workshop/Dataset/A/output.104 ground truth image(s) found.


In [13]:
IMAGE_COUNT = 300

p.sample(IMAGE_COUNT)

Processing <PIL.Image.Image image mode=RGBA size=256x256 at 0x7FCA444620B8>: 100%|██████████| 300/300 [00:06<00:00, 45.85 Samples/s]


In [ ]:
# Wait a few minutes before continuing to make sure the output images get uploaded to Google Drive, since we'll be 
# copying and combining them in the next step

In [16]:
newPathA = '/content/drive/My Drive/DigitalFutures Workshop/Dataset/Augmented/A/'
newPathB = '/content/drive/My Drive/DigitalFutures Workshop/Dataset/Augmented/B/'
combinedPath = '/content/drive/My Drive/DigitalFutures Workshop/Dataset/Augmented/Combined/'

In [17]:
# Iterate over the files generated by the Augmentor pipeline and copy them to a new directory
# The empty destination directory needs to exist before running this cell

# This is the default location that Augmentor will have saved the sampled images
os.chdir(pathA + '/output')

A_files = []
B_files = []

for filename in os.listdir('.'):
  if 'A_original' in filename:
      # guid will include the .png suffix
      guid = filename[-16:]
      A_files.append(guid)
      # shutil.copyfile(src, dst, *, follow_symlinks=True)
      dst = newPathA + guid
      shutil.copyfile(filename, dst)
  if '_groundtruth_' in filename:
      # guid will include the .png suffix
      guid = filename[-16:]
      B_files.append(guid)
      # shutil.copyfile(src, dst, *, follow_symlinks=True)
      dst = newPathB + guid
      shutil.copyfile(filename, dst)

print("A Filenames Found: ", A_files)
print("B Filenames Found: ", B_files)

A Filenames Found:  ['7495206b298b.png', '61bf81fcbe66.png', 'e66ba62a7fe4.png', '40d0366ab8e1.png', '1a955fdcf958.png', '3aa341e8a508.png', 'a9608243e385.png', 'e0e98edc2832.png', '3e1eed330262.png', '99945d124ee7.png', '4e2f76c0947c.png', '0144ad35e445.png', '3de9bfd55bc2.png', 'bf3d93193710.png', '62a78e2f8779.png', '6482e2ee255e.png', '48ce073539af.png', '4e8433295826.png', '8c57e14b4974.png', 'a6c56f97a1d6.png', '65e64625e617.png', 'de3901d3ade6.png', 'ba81a2df7c18.png', '72de3149100c.png', '98618b2b4d96.png', 'f4d8355d8b1c.png', 'fa3bf83d4c62.png', 'a27c54fca174.png', '75f7b9eb7a7a.png', '913b2297f09e.png', 'cfd51ce7dbee.png', 'afc8491643d9.png', '078d1f5c5cea.png', '92e9151853fb.png', 'b617a14bafdc.png', '11ef5ec2cbca.png', 'f048533019c7.png', 'ea1d46ebd5a4.png', '269cbad7f975.png', '2503be592a45.png', '3076f3975d7f.png', '94ae7606f9d8.png', 'b5aa993c2729.png', '6eb146f69d54.png', 'b0de1ee6054e.png', 'ce218e51b10b.png', '4dff5e77b04b.png', '99f5bf56f20e.png', '0800f1ca82d6.png',

In [27]:
# Combine Augmented Images into the pair format that Pix2Pix expects
os.chdir(newPathA)
for filename in os.listdir('.'):
  im1 = Image.open(filename)
  im2 = Image.open(newPathB + filename)

  # Combine images horizontally using PIL
  dst = Image.new('RGB', (512, 256))
  dst.paste(im1, (0, 0))
  dst.paste(im2, (im1.width, 0))
  dst.save(combinedPath + filename)

In [29]:
# Train/Test Split
# Takes the files in '../Augmented/Combined/' and randomly splits them into './Combined/Train' and './Combined/Test' directories
os.chdir(combinedPath)
test_dir = combinedPath + 'Test/'
train_dir = combinedPath + 'Train/'

files = os.listdir(combinedPath)
ratio = 0.15

for f in files:
    if '.png' in f:
      if np.random.rand(1) < ratio:
          shutil.copyfile(combinedPath + f, test_dir + f)
      else:
          shutil.copyfile(combinedPath + f, train_dir + f)

In [31]:
# Check that files exist in new location:

# os.chdir(pathA+ '/output')
os.chdir(train_dir)
print(len([name for name in os.listdir('.') if os.path.isfile(name)]))

252
